In [1]:
import numpy as np
import pandas as pd
# from scipy import signal
import matplotlib.pyplot as plt
import ipywidgets

In [2]:
from matplotlib import rcParams
rcParams["font.size"]=14

In [3]:
#import handcalcs.render

In [4]:
from math import sqrt, pi

In [5]:
import datetime as dt
import copy
from pandas.plotting import register_matplotlib_converters
from sklearn import linear_model
from sklearn.metrics import r2_score
import copy
import os
import matplotlib.backends.backend_pdf
lm = linear_model.LinearRegression()
np.random.seed(50)
plt.rcParams.update({'figure.max_open_warning': 0})
register_matplotlib_converters()
os.getcwd()

'G:\\My Drive\\DryCreek'

In [6]:
# local functions
import sys
sys.path.insert(1, './Functions')

from Abnormal_data_values_control import outvalues,threC
from Source_QC_functions_EM import interpolateQC, regressorFunc, funcClimateCatalogWg

In [7]:
# Only need to assign the two values below first, then run all
watershed = 'DryCreek'
main_str = 'LG'

In [8]:
# Read the original data table
table = pd.read_csv('1_'+watershed+'_Download_Aggregation.csv',header = 0,index_col = 'DateTime',
                    parse_dates = True, infer_datetime_format = True,low_memory=False)
display(table.head(2))
display(table.tail(2))

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.218182,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Check all the station names
all_stations = table.columns.str.extract(r'([^_]+)')[0]
print('All stations names: ', all_stations.unique())
print ('  ')
nameStrflwStation=[]
nameHydrMetStation=[]
for i in np.arange(len(table.columns)):
    if table.columns[i][-9:]=='Discharge':  ### 
        if not all_stations[i] in nameStrflwStation:
            nameStrflwStation.append(all_stations[i]) ### 
    else:
        if not all_stations[i] in nameHydrMetStation:
            nameHydrMetStation.append(all_stations[i])  ### 

print('Discharge stations :',nameStrflwStation)
print('  ')
print('Meteorology stations:',nameHydrMetStation)                  

All stations names:  ['TL' 'BSG' 'LG' 'C1E' 'C1W' 'C2E' 'C2M' 'BRW' 'LDP' 'SCR' 'LW' 'HN' 'HS'
 'MHN' 'MHS' 'MLN' 'MLS' 'LN' 'LS']
  
Discharge stations : ['TL', 'BSG', 'LG', 'C1E', 'C1W', 'C2E', 'C2M']
  
Meteorology stations: ['BRW', 'LDP', 'SCR', 'TL', 'LW', 'HN', 'HS', 'MHN', 'MHS', 'MLN', 'MLS', 'LN', 'LS']


In [10]:
for i in table.columns:
    if i.split('_')[0] == main_str :
        print(i)

LG_Discharge


#  1. Trim the original table

In [11]:
# Trim the data and let the table start with first non nan value, end with last non nan value
# Identify the first non nan value in each column as the starting date.
# Identify the last non nan value in each column as the ending date.

t = table.notna() 
t = ~np.isnan(table)
col = len(t.columns)
b = np.zeros([table.shape[1]])
c = np.array([table.shape[0]] * table.shape[1])

for i in range(col):
    if any(t.iloc[:,i]): # Since some are empty
        b[i] = list(np.where(t.iloc[:,i] == True))[0][0] # the first non nan value location
        c[i] = list(np.where(t.iloc[:,i] == True))[0][-1] # the last non nan value location
        
st_tab = b.min()
table1 = table.iloc[int(b.min()):int(c.max()) + 1,:] 

# Display the trimmed table
display(table1.head(2))
display(table1.tail(2))
print('trimmed row number is ', int(table.shape[0] -  table1.shape[0]))

,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
1999-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1999-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,TL_Discharge,BSG_Discharge,LG_Discharge,C1E_Discharge,C1W_Discharge,C2E_Discharge,C2M_Discharge,BRW_Precipitation,LDP_Precipitation,SCR_Precipitation,...,LS_SoilTemperature_Pit1_30cm,LS_SoilTemperature_Pit2_2cm,LS_SoilTemperature_Pit2_15cm,LS_SoilTemperature_Pit2_30cm,LS_SoilTemperature_Pit3_2cm,LS_SoilTemperature_Pit3_15cm,LS_SoilTemperature_Pit3_30cm,LS_SoilTemperature_Pit4_2cm,LS_SoilTemperature_Pit4_15cm,LS_SoilTemperature_Pit4_30cm
DateTime,,,,,,,,,,,,,,,,,,,,,
2020-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.218182,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.108333,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


trimmed row number is  0


In [12]:
table1.columns#[1:100]

Index(['TL_Discharge', 'BSG_Discharge', 'LG_Discharge', 'C1E_Discharge',
       'C1W_Discharge', 'C2E_Discharge', 'C2M_Discharge', 'BRW_Precipitation',
       'LDP_Precipitation', 'SCR_Precipitation',
       ...
       'LS_SoilTemperature_Pit1_30cm', 'LS_SoilTemperature_Pit2_2cm',
       'LS_SoilTemperature_Pit2_15cm', 'LS_SoilTemperature_Pit2_30cm',
       'LS_SoilTemperature_Pit3_2cm', 'LS_SoilTemperature_Pit3_15cm',
       'LS_SoilTemperature_Pit3_30cm', 'LS_SoilTemperature_Pit4_2cm',
       'LS_SoilTemperature_Pit4_15cm', 'LS_SoilTemperature_Pit4_30cm'],
      dtype='object', length=355)

### Quantity of interest

In [13]:
QOI = 'LS_SoilTemperature_Pit2_2cm'

In [14]:
table[QOI].interpolate(limit=7)

DateTime
1999-01-01   NaN
1999-01-02   NaN
1999-01-03   NaN
1999-01-04   NaN
1999-01-05   NaN
              ..
2020-05-07   NaN
2020-05-08   NaN
2020-05-09   NaN
2020-05-10   NaN
2020-05-11   NaN
Name: LS_SoilTemperature_Pit2_2cm, Length: 7802, dtype: float64

### 1. Interpolation

In [15]:
def widgetInterpolation(table, QOI, interplimit, Intpmethod, Intplimit_direction,
                        xmin=None, xmax=None, ymin=None, ymax=None):
    
    y = table[QOI]
     
    ind = np.where(~np.isnan(y)) # index where the value of y1 are not NaN
    lenNonNan = np.shape(ind)[1]  # number of non nan data points 
    totLen = np.shape(y)[0] # record length
    
    print('Total Record Length = ',totLen , ', Length of non NAN =', lenNonNan, '\n')
    
    if np.shape(y)[0] > lenNonNan:
        print('There are ', totLen - lenNonNan, ' missing values.', '\n' )
    yIntp = y.interpolate(limit=interplimit, method=Intpmethod,limit_direction=Intplimit_direction)
    
    # location where interpolation happened
   
    ind_interp = np.argwhere( np.isnan(y[:].to_numpy())!=  np.isnan(yIntp.to_numpy())  )
      
    indAfterInter = np.where(~np.isnan(yIntp)) # index where the value of y1 are not NaN - after interpolation
    lenNonNanAfter = np.shape(indAfterInter)[1]  # number of non nan data points after interpolation
    
    print('Length of interpolated values = ', lenNonNanAfter - lenNonNan, '\n')
    
    # Evaluate whether the interpolated values are out of the original range
    
    if np.max(yIntp) > np.max(y) or np.min(yIntp) < np.min(y):
        print("Interpolated values are out of the original data range.")
    else:
        print("No Interpolated values are out of the original data range.")
    
    return y,yIntp

    

In [16]:
a,b = widgetInterpolation(table1, QOI, interplimit=7, Intpmethod='time', Intplimit_direction='both')

Total Record Length =  7802 , Length of non NAN = 2502 

There are  5300  missing values. 

Length of interpolated values =  679 

No Interpolated values are out of the original data range.


In [17]:
pd.__version__

'1.0.1'

In [18]:
pd.datetime(2012,1,1)

C:\Users\edomm\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  """Entry point for launching an IPython kernel.


datetime.datetime(2012, 1, 1, 0, 0)

In [19]:
def Date_to_float(dt_time):
    return dt_time.year + (30*(dt_time.month-1) + dt_time.day)/367

In [20]:
def plot_widgetInterpolation(QOI=QOI, interplimit=7, Intpmethod='time', Intplimit_direction='both',
                        xmin=None, xmax=None, ymin=None, ymax=None):
    
    y, yIntp = widgetInterpolation(table1,QOI, interplimit, 'time', 'both')
    
    indx = Date_to_float(y.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, yIntp,'r',linewidth=2,label='Interpolated')
    ax.plot(indx, y, 'b',linewidth=2,label='Original')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

In [21]:
# plot_widgetInterpolation(QOI=QOI, interplimit=7, Intpmethod='time', Intplimit_direction='both',
#                         xmin=None, xmax=None, ymin=None, ymax=None)

In [22]:
%matplotlib notebook
# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook 
# %matplotlib widget - for jupyter lab
Interpolation_widget = ipywidgets.interactive(
    plot_widgetInterpolation,
    
    QOI=table1,
    Intpmethod = ['linear','time','space','index','pad','nearest', 'zero', 'slinear', 'quadratic', 'cubic'],
    Intplimit_direction=['forward', 'backward', 'both'],
    
    interplimit=ipywidgets.IntSlider(min=2, max=100, value=7),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
Interpolation_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…

### 2. Regression

In [23]:
# poool a variable from the input table based on QOI split -> into new table
# Perform regression for the QOI column

def widgetRegression(table, QOI, RegThreshold,xmin=None, xmax=None, ymin=None, ymax=None):
    
    
    col_names = pd.Series(table1.columns)
    varName = QOI.split('_')[1]

    df = copy.deepcopy(table[QOI].to_frame())
    df2 = copy.deepcopy(table[QOI].to_frame())
    df2[QOI] = np.nan
    
    
    ind = np.where(~np.isnan(table[QOI])) # index where the value of y1 are not NaN
    lenNonNan = np.shape(ind)[1]  # number of non nan data points 
    totLen = np.shape(table[QOI])[0] # record length
    
    print('Total Record Length = ',totLen , ', Length of non NAN =', lenNonNan, '\n')
    if np.shape(table[QOI])[0] > lenNonNan:
        print('There are ', totLen - lenNonNan, ' missing values.', '\n' )
    
    # print(np.shape(df.index))
    for i in range(len(col_names)):
        if QOI.split('_')[1] == col_names[i].split('_')[1]:
            #print(i, np.shape(table1[table1.columns[i]]), np.shape(df[QOI]))
            df.loc[df.index,col_names[i]] = table.loc[df.index,col_names[i]]

    print('Number of potential predictors = ',len(df.columns)-1,'\n')
    regR = regressorFunc(df,regThres=RegThreshold)
    # Location where regression is perfored
    ind_regression = np.argwhere(regR.reshape(len(regR)) != df.iloc[:,0].to_numpy())
    
    #print('Total record length =',np.shape(df)[0],'\n')
    print('Number of regressions performed =',len(ind_regression),'\n')
    
    if np.max(regR) > np.max(df[QOI]) or np.min(regR) < np.min(df[QOI]):
        print("Regression values are out of the original data range.")
    else:
        print("No Regression values are out of the original data range.")
          
    df2[QOI] = regR
    
    return df[QOI],df2[QOI]



In [24]:
rawdd, regRes = widgetRegression(table1, QOI, RegThreshold=1.0,xmin=None, xmax=None, ymin=None, ymax=None)

Total Record Length =  7802 , Length of non NAN = 2502 

There are  5300  missing values. 

Number of potential predictors =  155 

Number of regressions performed = 5300 

No Regression values are out of the original data range.


In [25]:
def plot_widgetRegression(QOI=QOI, RegThreshold=0.7, xmin=None, xmax=None, ymin=None, ymax=None):
    
    y, yReg = widgetRegression(table1, QOI, RegThreshold=0.7)
    
    indx = Date_to_float(y.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, yReg,'r',linewidth=2,label='After Regression')
    ax.plot(indx, y, 'b',linewidth=2,label='Raw Data')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()
    # fig.gcf().canvas.set_window_title('Regression')

In [26]:
#plot_widgetRegression(QOI=QOI, RegThreshold=0.7, xmin=None, xmax=None, ymin=None, ymax=None)

In [27]:
%matplotlib notebook
# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook 
# %matplotlib widget - for jupyter lab
Regression_widget = ipywidgets.interactive(
    plot_widgetRegression,
    
    QOI=table1,
    
    RegThreshold=ipywidgets.FloatSlider(min=.5, max=1, value=7),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
Regression_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…

### 3. Climate catalog

In [28]:
#CC_Yout, yraw, indx_CC= funcClimateCatalogWg(table,QOI,thrLen=20,corrThr=0.7)

In [29]:
# %matplotlib inline
# plt.plot(CC_Yout,'b')
# plt.plot(yraw,'r')

In [30]:
def plot_widgetClimateCatalog(QOI=QOI, thrLen=200, corrThr=0.7, xmin=None, xmax=None, ymin=None, ymax=None):
    
    y_CC, yraw, indx_cc = funcClimateCatalogWg(table1, QOI, thrLen, corrThr)
    
    indx = Date_to_float(yraw.index)
    
    fig, ax = plt.subplots(1, 1, figsize=(9, 4))
    ax.plot(indx, y_CC,'r',linewidth=2,label='After Climate Catalog')
    ax.plot(indx, yraw, 'b',linewidth=2,label='Raw Data')
    
    ax.set_xlabel("Year")
    ax.set_ylabel(QOI.split('_')[1])
    ax.set_title(QOI)
    ax.set_xlim([xmin,xmax])
    ax.set_ylim([ymin, ymax])
    ax.grid(color='gray', linestyle='-.', linewidth=1)
    #ax.legend(bbox_to_anchor=(1, 1), loc='upper left', fontsize='small')
    ax.legend(fontsize='small')
    
    # interaction responsiveness
    ax.relim()
    ax.autoscale_view()
    fig.canvas.draw_idle()

In [31]:
#plot_widgetClimateCatalog(QOI=QOI, thrLen=100, corrThr=0.7, xmin=None, xmax=None, ymin=None, ymax=None)

In [32]:
%matplotlib notebook
# %matplotlib qt, - a separate plot
# %matplotlib notebook - the plot reside within the notebook 
# %matplotlib widget - for jupyter lab
ClimateCatalog_widget = ipywidgets.interactive(
    plot_widgetClimateCatalog,
    
    QOI=table1,
    
    corrThr=ipywidgets.FloatSlider(min=.5, max=1, value=7),
    thrLen=ipywidgets.IntSlider(min=1, max=365, value=7),
    xmin=ipywidgets.FloatText(value=2008),
    xmax=ipywidgets.FloatText(value=2018),
    ymin=ipywidgets.FloatText(value=0),
    ymax=ipywidgets.FloatText(value=50)
)
ClimateCatalog_widget

interactive(children=(Dropdown(description='QOI', index=346, options=('TL_Discharge', 'BSG_Discharge', 'LG_Dis…

In [ ]:
## VErsions of packages used
# print(sklearn.__version__)
# pip3 list | findstr scikit
# conda list |findstr pandas
# pip list | findstr "scikit numpy"